In [ ]:
import os
from bedrock_agentcore_starter_toolkit import Runtime

# 1. Clear any old AWS credentials
os.environ.pop("AWS_ACCESS_KEY_ID", None)
os.environ.pop("AWS_SECRET_ACCESS_KEY", None)
os.environ.pop("AWS_SESSION_TOKEN", None)

# 2. Set your SSO profile, here I'm utilizing aws sso login
os.environ["AWS_PROFILE"] = "sandbox"  


# 4. Initialize Runtime
agentcore_runtime = Runtime()

# 5. Configure the agent
agent_name = "ticker-checker-agent"
response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py", 
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region="us-east-1",
    agent_name=agent_name
)

print("AgentCore configuration response:")
print(response)

Entrypoint parsed: file=C:\Users\machupichu\Desktop\Repo\agentcore\langgraph_bedrock.py, bedrock_agentcore_name=langgraph_bedrock


Configuring BedrockAgentCore agent: langgraph_claude_getting_started


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: c:\Users\machupichu\Desktop\Repo\agentcore\Dockerfile
Generated .dockerignore: c:\Users\machupichu\Desktop\Repo\agentcore\.dockerignore
Keeping 'langgraph_claude_getting_started' as default agent
Bedrock AgentCore configured: c:\Users\machupichu\Desktop\Repo\agentcore\.bedrock_agentcore.yaml


AgentCore configuration response:
config_path=WindowsPath('c:/Users/machupichu/Desktop/Repo/agentcore/.bedrock_agentcore.yaml') dockerfile_path=WindowsPath('c:/Users/machupichu/Desktop/Repo/agentcore/Dockerfile') dockerignore_path=WindowsPath('c:/Users/machupichu/Desktop/Repo/agentcore/.dockerignore') runtime='None' region='us-east-1' account_id='376945888930' execution_role=None ecr_repository=None auto_create_ecr=True


In [13]:
launch_result = agentcore_runtime.launch()


🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'langgraph_claude_getting_started' to account 376945888930 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: langgraph_claude_getting_started
✅ ECR repository available: 376945888930.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-langgraph_claude_getting_started


✅ Reusing existing ECR repository: 376945888930.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-langgraph_claude_getting_started


Getting or creating execution role for agent: langgraph_claude_getting_started
Using AWS region: us-east-1, account ID: 376945888930
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-2292fc8165
✅ Reusing existing execution role: arn:aws:iam::376945888930:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-2292fc8165
✅ Execution role available: arn:aws:iam::376945888930:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-2292fc8165
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: langgraph_claude_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-2292fc8165
Reusing existing CodeBuild execution role: arn:aws:iam::376945888930:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-2292fc8165
Using .dockerignore with 44 patterns
Uploaded source to S3: langgraph_claude_getting_started/source.zip
Updated CodeBuild project: bedrock-agentcore-langgraph_claude_getting_started-builder
Starting CodeBuild build (this may take s

In [6]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: langgraph_claude_getting_started


'READY'

In [14]:
invoke_response = agentcore_runtime.invoke({"prompt": "Whats the btc price today"})
invoke_response

{'ResponseMetadata': {'RequestId': '9fb26993-5102-4710-a10b-a02f74694d76',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 27 Oct 2025 13:04:52 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9fb26993-5102-4710-a10b-a02f74694d76',
   'baggage': 'Self=1-68ff6def-08c466482ed56c5f480f0987,session.id=c96cd2d3-0200-4aaf-9709-3a6889298713',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'c96cd2d3-0200-4aaf-9709-3a6889298713',
   'x-amzn-trace-id': 'Root=1-68ff6def-7558683849c8e9666b3217db;Parent=9b993e19f4286a66;Sampled=1;Self=1-68ff6def-08c466482ed56c5f480f0987'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'c96cd2d3-0200-4aaf-9709-3a6889298713',
 'traceId': 'Root=1-68ff6def-7558683849c8e9666b3217db;Parent=9b993e19f4286a66;Sampled=1;Self=1-68ff6def-08c466482ed56c5f480f0987',
 'baggage': 'Self=1-68ff6def-08c466482ed56c5f480f0987,session.id=c96cd2d3-0200-4aaf-9709-3a6889298713',
 'contentType': 

In [15]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

I apologize, but it seems there's a technical issue with the tool that fetches the Bitcoin price. The system is unable to connect to the price data source at the moment.

Would you like me to try again, or is there something else I can help you with today?

In [9]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name="us-east-1"
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The current weather is sunny! It's a beautiful day outside.

In [10]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]


('376945888930.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-langgraph_claude_getting_started',
 'langgraph_claude_getting_started-IQOqbL5C0U',
 'bedrock-agentcore-langgraph_claude_getting_started')